In [37]:
#from newencoder import Encoder
from newencoder import Encoder
import torch
import numpy as np
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
W2V_PATH = "/home/jingjing/Desktop/InferSent-master/dataset/GloVe/glove.840B.300d.txt"

In [25]:
f = Encoder()
f.set_w2v_path(W2V_PATH)

f.zero_grad()

for i in f.parameters():
    i.retain

True
None
True
None
True
None
True
None
True
None
True
None
True
None
True
None


In [26]:
def make_target(context_size, dim):
    targets = np.zeros((dim, dim))
    ctxt_sent_pos = list(range(-context_size, context_size+1))
    ctxt_sent_pos.remove(0)
    for ctxt in ctxt_sent_pos:
        targets += np.eye(3, k=ctxt)
    targets_sum = np.sum(targets,axis=1, keepdims=True)
    targets = targets / targets_sum
    targets = torch.from_numpy(targets)
    return targets

In [27]:
def make_senmat(enc, sentences, bsize, tokenize, verbose):
    enc.build_vocab(sentences, True)
    embeddings = enc.encode(sentences, bsize, tokenize, verbose)
    scores = np.matmul(embeddings,np.transpose(embeddings))
    scores_sum = np.sum(scores, axis=1, keepdims=True)
    scores = scores/scores_sum
    scores = torch.from_numpy(scores)
    scores.requires_grad = True
    return scores

In [28]:
def xentropy_cost(pred, target):
    logged = torch.log(pred)
    a = target.float()*logged
    cost = -torch.sum(a)
    return cost

In [29]:

def loss_fn(pred, target):
    m = nn.Softmax(dim=-1)
    s_pred = m(pred)
    losses = F.binary_cross_entropy_with_logits(s_pred, target)
    return losses

In [30]:
data = [['Memories of childhood are unforgettable.', 'I was four years old when my grandfather died.',
             'I clearly remember how everybody in the house was weeping.'], ['Today is sunny', 'We should go out for a picnic.', 'Love the weather.']]

In [31]:
test = ['Doctors lead a hard life.', 'Their life is very busy.', 'They get up early in the morning and go to the hospital.']

In [32]:
with torch.no_grad():
    f.build_vocab(test, True)
    scores = f(test, 400, False, True)
    #scores = make_senmat(f, test, 400, False, True)
    print(scores)
    targets = make_target(1, 3)
    print(targets)
    loss = nn.L1Loss(scores, targets.float())
    print(loss)

Found 23(/23) words with w2v vectors
Vocab size : 23
Nb words kept : 25/28 (89.3%)
Speed : 103.3 sentences/s (cpu mode, bsize=3)
tensor([[ 0.3544,  0.3179,  0.3277],
        [ 0.3212,  0.3540,  0.3249],
        [ 0.3252,  0.3191,  0.3557]])
tensor([[ 0.0000,  1.0000,  0.0000],
        [ 0.5000,  0.0000,  0.5000],
        [ 0.0000,  1.0000,  0.0000]], dtype=torch.float64)
L1Loss()


In [33]:
optimizer = optim.Adam(f.parameters(), lr=0.1)


In [36]:
for epoch in range(2):
    for instance in data:
        f.zero_grad()
        scores = f(instance, 400, False, False)
        scores = Variable(scores, requires_grad=True)
        #scores = make_senmat(f, instance, 400, False, True)
        
        targets = make_target(1,3)
        loss_f = nn.L1Loss()
        loss = loss_f(scores, targets.float())
        print(epoch, loss)
        loss.backward()
        optimizer.step()
        print(list(f.parameters())[0].grad)
        print(scores.grad)

0 tensor(0.3671)
None
tensor([[ 0.1111, -0.1111,  0.1111],
        [-0.1111,  0.1111, -0.1111],
        [ 0.1111, -0.1111,  0.1111]])
0 tensor(0.3718)
None
tensor([[ 0.1111, -0.1111,  0.1111],
        [-0.1111,  0.1111, -0.1111],
        [ 0.1111, -0.1111,  0.1111]])
1 tensor(0.3671)
None
tensor([[ 0.1111, -0.1111,  0.1111],
        [-0.1111,  0.1111, -0.1111],
        [ 0.1111, -0.1111,  0.1111]])
1 tensor(0.3718)
None
tensor([[ 0.1111, -0.1111,  0.1111],
        [-0.1111,  0.1111, -0.1111],
        [ 0.1111, -0.1111,  0.1111]])


In [16]:
for name, params in f.named_parameters():
    print(name)

enc_gru.weight_ih_l0
enc_gru.weight_hh_l0
enc_gru.bias_ih_l0
enc_gru.bias_hh_l0
enc_gru.weight_ih_l0_reverse
enc_gru.weight_hh_l0_reverse
enc_gru.bias_ih_l0_reverse
enc_gru.bias_hh_l0_reverse


In [72]:
with torch.no_grad():
    scores = f(test, 400, False, True)
    print(scores)
    targets = make_target(1, 3)
    print(targets)
    loss = loss_fn(scores, targets.float())
    print(loss)

Nb words kept : 25/28 (89.3%)
Speed : 77.2 sentences/s (cpu mode, bsize=400)
tensor([[ 0.3526,  0.3160,  0.3315],
        [ 0.3181,  0.3541,  0.3279],
        [ 0.3233,  0.3176,  0.3591]])
tensor([[ 0.0000,  1.0000,  0.0000],
        [ 0.5000,  0.0000,  0.5000],
        [ 0.0000,  1.0000,  0.0000]], dtype=torch.float64)
tensor(0.8333)


In [26]:
g = Encoder()

In [27]:
g.set_w2v_path(W2V_PATH)